In [3]:
import pandas as pd
metadata = pd.read_csv('sbsspecial.csv', encoding='cp949')

In [4]:
metadata.head(3)

,Unnamed: 0,tv,title,episode,date,link,synopsys
0,0,SBS,SBS 스페셜,복흥 O.B.A 소녀들은 자란다,2018.09.16(일),http://programs.sbs.co.kr/culture/sbsspecial/v...,[ 복흥 O.B.A소녀들은 자란다 사춘기 자녀를 둔 부모들에게 아이를 키우는...
1,1,SBS,SBS 스페셜,자영업 공화국의 눈물,2018.09.09(일),http://programs.sbs.co.kr/culture/sbsspecial/v...,[ 자영업 공화국의 눈물 대한민국의 자영업자 570만 명.경제협력개발기구(OECD)...
2,2,SBS,SBS 스페셜,다시 쓰는 불면일기 잠은 죽어서 자나?!,2018.09.02(일),http://programs.sbs.co.kr/culture/sbsspecial/v...,[ ??다시 쓰는 불면일기잠은 죽어서 자나?!잠을 팔고 잠을 사는 사회 ...


In [27]:
metadata['episode'].head()

0         복흥 O.B.A 소녀들은 자란다
1               자영업 공화국의 눈물
2    다시 쓰는 불면일기 잠은 죽어서 자나?!
3                아이와 여행하는 법
4                 영미네 작은 식탁
Name: episode, dtype: object

In [6]:
metadata['synopsys'].head()

0    [ 복흥 O.B.A소녀들은 자란다 사춘기 자녀를 둔 부모들에게     아이를 키우는...
1    [ 자영업 공화국의 눈물 대한민국의 자영업자 570만 명.경제협력개발기구(OECD)...
2    [ ??다시 쓰는 불면일기잠은 죽어서 자나?!잠을 팔고 잠을 사는 사회       ...
3    [ 아이와 여행하는 법▶ 여행에도 방법이 있다?!?나는 아이와 몇 점짜리 여행을 하...
4    [ 영미네 작은 식탁방송일시: 2018년 8월 5일(일) 밤 11시 5분연출: 황성...
Name: synopsys, dtype: object

### TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='ko')
metadata['synopsys'] = metadata['synopsys'].fillna('')

In [18]:
#tfidf_matrix = tfidf.fit_transform(metadata['synopsys'])

tfidf_vectorizer = TfidfVectorizer(min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(metadata['synopsys'])

In [16]:
tfidf_matrix.shape

(501, 12228)

### 코사인 유사성(코사인 각이 유사도 결정)

from sklearn.metrics.pairwise import linear_kernel

#코사인 유사도 매트릭스 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

### 에피소드 제목과 시놉시스 역 매핑? 

In [21]:
indices = pd.Series(metadata.index, index=metadata['episode'])

In [26]:
#입력은 에피소드 제목 출력은 비슷한 에피소드

def get_recommendations(episode, cosine_sim=cosine_sim):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[episode]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
    return metadata['episode'].iloc[epi_indices]

In [34]:
get_recommendations('아이와 여행하는 법')

66                    중년의 사생활, 갱년기
43                          헌법의 탄생
68               금연전쟁 - 내 아이를 부탁해 
103    암(癌) 전문의의 선택 진료실에서 못다 한 이야기
234                      노엘의 귀향 2부
93             설탕전쟁 - 당(糖)하고 계십니까?
223                 동물, 행복의 조건 1부 
67              세월의 가치를 더하다 - 세운상가
230                   -화무십일홍 권불십년-
0                복흥 O.B.A 소녀들은 자란다
Name: episode, dtype: object

In [ ]:
metad